# 19.6 Takeaway - Readers Writers First Problem

Consider an object s which is read from and written to by many threads. You need to ensure that no thread may access s for reading or writing while another thread is writing to s. Two or more readers may access s at the same time.

Implement a synchronization mechanism for this readers-writers problem in python without busy waiting. 

## Brainstorming

- We need 1 ReaderWriter class that holds two locks. LR for the read lock, LW for the write lock.
- Have an int called `data` as an attribute of the ReaderWriter class.
- Have another int for a `read_count` that counts how many read threads are active.
- Need 1 class for a Reader Thread
- Need 1 class for a Writer Thread

In [25]:
import threading
import time

class ReaderWriter:
    def __init__(self):
        self.LR = threading.Condition()
        self.LW = threading.Condition()
        self.read_count = 0
        self.data = 0

class Reader(threading.Thread):
    def __init__(self, rw):
        super().__init__()
        self.rw = rw

    def run(self):
        # Removing while True to test this properly
        #while True:
            with self.rw.LR:
                self.rw.read_count += 1

            print(f'Reader {self.name} reading data {self.rw.data}')
            with self.rw.LR:
                self.rw.read_count -= 1
                self.rw.LR.notify()

class Writer(threading.Thread):
    def __init__(self, rw):
        super().__init__()
        self.rw = rw

    def run(self):
        # Removing while True to test this properly
        # while True: 
            with self.rw.LW:
                done = False
                while not done:
                    with self.rw.LR:
                        if self.rw.read_count == 0:
                            self.rw.data += 1
                            done = True
                        else:
                            self.rw.LR.wait()
                print(f'Writer {self.name} wrote data {self.rw.data}')



In [26]:
# Create a ReaderWriter instance
rw = ReaderWriter()

# Create some Reader and Writer threads
readers = [Reader(rw) for i in range(5)]
writers = [Writer(rw) for i in range(2)]

# Start all the threads
for t in readers:
    t.start()

for t in writers:
    t.start()

# Wait for all the threads to terminate
for t in readers + writers:
    t.join()

# Check that the data is correct
print(f'Final data value is {rw.data}')                

Reader Thread-141 reading data 0
Reader Thread-142 reading data 0
Reader Thread-143 reading data 0
Reader Thread-144 reading data 0
Reader Thread-145 reading data 0
Writer Thread-146 wrote data 1
Writer Thread-147 wrote data 2
Final data value is 2


## Problem with the First Readers Writers Problem

The implementation of the first reader-writer problem that uses a counter to keep track of the number of readers in the critical section can potentially lead to starvation of writers.

The implementation gives priority to readers over writers. Specifically, the implementation ensures that any waiting reader thread can enter the critical section before any waiting writer thread can enter the critical section, even if the writer thread has been waiting longer. This can cause a situation where a writer is waiting indefinitely, while readers keep accessing the critical section.

Therefore, if the system has a high number of readers, this implementation could potentially starve writers.

# 19.7 Takeaway - With Writer Preference aka 2nd Readers Writer Problem

We can try giving writers the preference because of the above problem.

In [27]:
import threading
import time
import random

class ReaderWriter:

    def __init__(self):
        self.data = 0
        self.read_count = 0
        self.write_count = 0
        self.lock = threading.Lock()
        self.can_read = threading.Condition(self.lock)
        self.can_write = threading.Condition(self.lock)

    def read(self, name):
        with self.can_read:
            # Wait until there are no writers
            while self.write_count > 0:
                self.can_read.wait()

            # Increment read count
            self.read_count += 1

        # Read data
        print(f'{name} reading {self.data}')

        with self.can_read:
            # Decrement read count
            self.read_count -= 1

            # Notify waiting writers if no readers
            if self.read_count == 0:
                self.can_write.notify()

    def write(self, name):
        with self.can_write:
            # Wait until there are no readers or writers
            while self.read_count > 0 or self.write_count > 0:
                self.can_write.wait()

            # Increment write count
            self.write_count += 1

        # Write data
        self.data += 1
        print(f'{name} writing {self.data}')

        with self.can_write:
            # Decrement write count
            self.write_count -= 1

            # Notify waiting readers or writers
            self.can_write.notify()
            self.can_read.notify()

In [28]:
# Test the implementation
def test():
    rw = ReaderWriter()

    # Start reader threads
    for i in range(5):
        t = threading.Thread(target=rw.read, args=(f"reader {i}",))
        t.start()

    # Start writer threads
    for i in range(2):
        t = threading.Thread(target=rw.write, args=(f"writer {i}",))
        t.start()

    '''
    # Wait for all threads to finish
    for t in threading.enumerate():
        t.join()
    '''

test()

reader 0 reading 0
reader 1 reading 0
reader 2 reading 0
reader 3 reading 0
reader 4 reading 0
writer 0 writing 1
writer 1 writing 2
